# Community Detection in a Facebook network

## Dataset

In [1]:
import networkx as nx

In [2]:
edges_filename = '0.edges'

In [3]:
# Load the Facebook dataset
graph = nx.read_edgelist(f'data/{edges_filename}')

In [ ]:
# Explore the properties of the graph

In [4]:
# print number of nodes / edges
print('Number of nodes:', graph.number_of_nodes())
print('Number of edges:', graph.number_of_edges())

Number of nodes: 333
Number of edges: 2519


## Algorithms implementation

In [5]:
# Implement the Louvain method for community detection.

In [6]:
# Implement the average clustering coefficient for a graph.

## Analysis

In [7]:
# Identify users’ communities in the Facebook network using Louvain

In [ ]:
# Identify the top k users with the highest clustering coefficient in the graph.
# Experiment with different values of k and choose the most appropriate one.

In [ ]:
# Evaluate different random walk strategies to spread a message across the network.
# The message should reach as many different communities as possible.

## Visualization

In [8]:
# Visualize the output of Louvain by coloring the nodes according to their assigned Communities.

In [ ]:
# Visualize the output of applying Random Walk algorithm, by highlighting the sequence of nodes selected in a path.